In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("data/commercial_area.csv", dtype={"행정구역코드": "object"})

In [3]:
# 시군구 추출
area_code = {k: v for v, k in df[df["구분"] == "gu"][["행정구역", "행정구역코드"]].values}

# 행정동 추출
df = df[df["구분"] == "dong"].reset_index(drop=True)
df["시군구"] = df["행정구역코드"].apply(lambda x: area_code[x[:5]])

df.drop(["행정구역코드", "구분"], axis=1, inplace=True)
df2 = df[[df.columns[-1]] + list(df.columns[:-1])]

# 값 순위

In [4]:
ranked = df2.set_index(['시군구','행정구역']).groupby('시군구').rank(method='dense',ascending=False).reset_index()

In [5]:
ranked = ranked.rename(columns={column:column+' 구내 순위' for column in ranked.columns if column not in ['시군구','행정구역']})

# 변화량 순위

In [6]:
change = df2[[column for column in df2.columns if '소득' not in column]]

In [7]:
change = change.replace('-',0).replace(' ',0)

In [8]:
change.iloc[:,2:] = change.iloc[:,2:].astype(float)

In [9]:
for column in change.columns:
    if '2022' in column:
        change[column+' 작년 동분기 대비 상승률'] = round(100*(change[column]-change[column.replace('2022','2021')])/change[column.replace('2022','2021')],1)
        change[column+' 작년 동분기 대비 하락률'] = round(100*(change[column.replace('2022','2021')]-change[column])/change[column.replace('2022','2021')],1)


In [10]:
change_ranked = change[['시군구','행정구역']+[column for column in change.columns if column[-1]=='률']].fillna(0).set_index(['시군구','행정구역']).groupby('시군구').rank(method='dense',ascending=False).reset_index()

In [11]:
change_ranked = change_ranked.rename(columns={column:column+' 구내 순위' for column in change_ranked.columns if column not in ['시군구','행정구역']})

# Merge

In [12]:
merged = df2.merge(ranked).merge(change_ranked)

In [13]:
merged = merged.astype({col: int for col in merged.select_dtypes(include='float64').columns})

In [15]:
merged.to_csv("data/preprocessed_commercial_area.csv", index=False)